In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from simpletransformers.language_modeling import LanguageModelingModel

from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import itertools
from collections import Counter

from tqdm import tqdm
import torch

import networkx as nx

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from functools import partial

import pickle

from collections import deque

from torchvision import models
import torch.nn.functional as F
from torch import nn

from sklearn.utils import resample

import fasttext
import sister



from typing import List
from pathlib import Path
from joblib import dump, load

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/home/roshansk/YelpAnalysis/')
from utils import *

import spacy

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')
# nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl_subword/')

sys.path.insert(0,'/data2/Datasets/')
from preprocess import *

dataFolder = '/data2/Datasets/Raw'

### Load Dataset

In [6]:
trainDf, devDf, testDf = splitData(getData(dataFolder, 'yelp_subset'))

In [7]:
trainData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')

testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')

trainDataset = Dataset(trainDf, trainData)
testDataset = Dataset(testDf, testData)

### Model Training

In [13]:
NNnet = trainFFN(trainDataset, testDataset, num_epochs = 3)

200it [00:00, 219.48it/s]


Acc : 0.869 F1 : 0.9054151624548736
*************************


200it [00:00, 314.05it/s]

Acc : 0.91 F1 : 0.9380165289256197
*************************



200it [00:00, 231.03it/s]

Acc : 0.896 F1 : 0.927170868347339
*************************


In [16]:
lexicon = generateLexicon_FFN(NNnet,trainDf, nlp, device ='cuda:0')

lexicon.rename({'NNprob':'score'},axis = 1, inplace = True)
lexiconWords, lexiconMap = getLexicon(0, df = lexicon)



In [18]:
testFFN(NNnet,'yelp_subset',lexiconWords, lexiconMap, nlp, dataFolder,  train = True)

2381it [00:07, 319.86it/s]


yelp_subset , 0.9387600806451613 , 0.9361366622864652 , 0.927 , 0.926


('yelp_subset', 0.9387600806451613, 0.9361366622864652, 0.927, 0.926)

### Testing

In [24]:
dataList = ['dialog','amazon_finefood_subset','amazon_toys_subset','yelp_subset','empathy','nrc_joy']
for data in dataList:
    testFFN(NNnet,data,lexiconWords, lexiconMap, nlp)

2708it [00:07, 363.19it/s]


dialog , 0.4027182744866302 , 0.4372216035634744 , 0.815 , 0.898


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

200it [00:00, 222.91it/s]


amazon_finefood_subset , 0.308 , 0.32289628180039137 , 0.85 , 0.919


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

200it [00:00, 244.60it/s]


...

yelp_subset , 0.385 , 0.3263964950711939 , 0.758 , 0.858


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

38it [00:00, 134.79it/s]


empathy , 0.489247311827957 , 0.0 , 0.505 , 0.607


0it [00:00, ?it/s]/home/roshansk/YelpAnalysis/utils.py:530: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.

421it [00:02, 206.69it/s]


nrc_joy , 0.7605700712589074 , 0.6589986468200271 , 0.724 , 0.635
